# Answering Business Questions using SQL
SQL is a powerful data analysis tool. 

Today, I will use SQL to analyze a sample database, Chinook. Chinook is a mock representation of a digital music media store.

I will assume that I work for the fictional digital music media company Chinook, and that I have been tasked with analyzing sales and marketing data for the company.

The database and information on the tables in the database can be found <a href='https://www.sqlitetutorial.net/sqlite-sample-database/'>here</a>.

Some valuable information about Chinook can be found <a href='https://github.com/lerocha/chinook-database/blob/master/README.md'>here</a>.

### Goal:

I will provide an answer for each of the following questions/demands:   
<ul><li>What 3 new albums should we add to the store, from a list of 4 (list provided)?</li>
    <li>Are any sales support agents performing better or worse than their peers?</li>
    <li>Calculate for each country: (a) total number of customers, (b) total value of sales, (c) average value of sales per customer, (d) average order value.</li>
    <li>What percentage of purchases are individual tracks vs. whole albums?</li>

## Connecting to SQL and the Database

In [1]:
%%capture
%load_ext sql

In [2]:
%sql sqlite:////Users/admin/Downloads/Databases/chinook.db

'Connected: @/Users/admin/Downloads/Databases/chinook.db'

## Viewing the Chinook Database

In [3]:
%%sql

SELECT *
  
    FROM sqlite_master
 
    WHERE type="table";

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,albums,albums,2,"CREATE TABLE ""albums"" ( [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Title] NVARCHAR(160) NOT NULL, [ArtistId] INTEGER NOT NULL, FOREIGN KEY ([ArtistId]) REFERENCES ""artists"" ([ArtistId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,artists,artists,4,"CREATE TABLE ""artists"" ( [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,customers,customers,5,"CREATE TABLE ""customers"" ( [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [FirstName] NVARCHAR(40) NOT NULL, [LastName] NVARCHAR(20) NOT NULL, [Company] NVARCHAR(80), [Address] NVARCHAR(70), [City] NVARCHAR(40), [State] NVARCHAR(40), [Country] NVARCHAR(40), [PostalCode] NVARCHAR(10), [Phone] NVARCHAR(24), [Fax] NVARCHAR(24), [Email] NVARCHAR(60) NOT NULL, [SupportRepId] INTEGER, FOREIGN KEY ([SupportRepId]) REFERENCES ""employees"" ([EmployeeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,employees,employees,8,"CREATE TABLE ""employees"" ( [EmployeeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [LastName] NVARCHAR(20) NOT NULL, [FirstName] NVARCHAR(20) NOT NULL, [Title] NVARCHAR(30), [ReportsTo] INTEGER, [BirthDate] DATETIME, [HireDate] DATETIME, [Address] NVARCHAR(70), [City] NVARCHAR(40), [State] NVARCHAR(40), [Country] NVARCHAR(40), [PostalCode] NVARCHAR(10), [Phone] NVARCHAR(24), [Fax] NVARCHAR(24), [Email] NVARCHAR(60), FOREIGN KEY ([ReportsTo]) REFERENCES ""employees"" ([EmployeeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,genres,genres,10,"CREATE TABLE ""genres"" ( [GenreId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,invoices,invoices,11,"CREATE TABLE ""invoices"" ( [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerId] INTEGER NOT NULL, [InvoiceDate] DATETIME NOT NULL, [BillingAddress] NVARCHAR(70), [BillingCity] NVARCHAR(40), [BillingState] NVARCHAR(40), [BillingCountry] NVARCHAR(40), [BillingPostalCode] NVARCHAR(10), [Total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([CustomerId]) REFERENCES ""customers"" ([CustomerId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items"" ( [InvoiceLineId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [InvoiceId] INTEGER NOT NULL, [TrackId] INTEGER NOT NULL, [UnitPrice] NUMERIC(10,2) NOT NULL, [Quantity] INTEGER NOT NULL, FOREIGN KEY ([InvoiceId]) REFERENCES ""invoices"" ([InvoiceId]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([TrackId]) REFERENCES ""tracks"" ([TrackId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,media_types,media_types,15,"CREATE TABLE ""media_types"" ( [MediaTypeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,playlists,playlists,16,"CREATE TABLE ""playlists"" ( [PlaylistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"


## Viewing the Tables

In [4]:
%%sql

SELECT
    
    name,
    
    type

        FROM sqlite_master

            WHERE type IN ("table","view");

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


name,type
albums,table
sqlite_sequence,table
artists,table
customers,table
employees,table
genres,table
invoices,table
invoice_items,table
media_types,table
playlists,table


## Viewing The Schema
The schema for the database looks like this:

<div style = 'width: 900px;'>
    <img src='https://cdn.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg'>
</div>

## Answering the Questions

### What 3 new albums should we add to the store?
There are 4 albums from a record label with a new record deal that Chinook could feature in its store.

The 4 album artists and album genres are:
<ul><li>Regal (Hip-Hop)</li>
    <li>Red Tone (Punk)</li>
    <li>Meteor and the Girls (Pop)</li>
    <li>Slim Jim Bites (Blues)</li></ul>
    
The record label specializes in artists from the USA, and they expect Chinook to advertise their artists heavily in the USA. 

Which 3 albums should Chinook feature in the store?

To answer this question, I will determine which 3 of these genres are more popular than the other in Chinook's USA market.

To do this, I will determine how many tracks of each of these genres are sold in the USA, and choose the 3 albums that correspond to the 3 most-selling genres.

In [5]:
%%sql

/* #Listing all genres sold in USA and number of tracks sold in USA */

SELECT 
    
    g.Name Genre, 
    
    COUNT(i.InvoiceId) Tracks 
        
        FROM customers c
        
        INNER JOIN invoices i ON i.CustomerId = c.CustomerId
        INNER JOIN invoice_items ii ON ii.InvoiceId = i.InvoiceId
        INNER JOIN tracks t ON t.TrackId = ii.TrackId
        INNER JOIN genres g ON g.GenreId = t.GenreId
        
            WHERE c.Country = "USA"
        
                GROUP BY Genre
        
                ORDER BY 2 DESC;

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


Genre,Tracks
Rock,157
Latin,91
Metal,64
Alternative & Punk,50
Jazz,22
Blues,15
TV Shows,14
R&B/Soul,12
Comedy,8
Classical,8


In [6]:
%%sql

/* #Isolating relevant genres */

SELECT 
    
    g.Name Genre, 
    
    COUNT(i.InvoiceId) Tracks 
        
        FROM customers c
        
        INNER JOIN invoices i ON i.CustomerId = c.CustomerId
        INNER JOIN invoice_items ii ON ii.InvoiceId = i.InvoiceId
        INNER JOIN tracks t ON t.TrackId = ii.TrackId
        INNER JOIN genres g ON g.GenreId = t.GenreId
        
            WHERE c.Country = "USA" 
        
            AND Genre IN ("Hip Hop/Rap", "Alternative & Punk", "Pop", "Blues")  /* #Isolating relevant genres */
        
                GROUP BY Genre
        
                ORDER BY 2 DESC;

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


Genre,Tracks
Alternative & Punk,50
Blues,15
Pop,5
Hip Hop/Rap,4


The Alternative & Punk, Blues, and Pop genres are all more popular than the Hip Hop/Rap genre in our USA market.

### Answer
Therefore, the three albums that we should add to the store are:   
<ul><li>Red Tone's punk album</li>
    <li>Slim Jim Bites' blues album</li>
    <li>Meteor and the Girls' pop album</li></ul>

### Are any sales support agents performing better or worse than their peers?
To answer this question I will calculate the total dollar amount (the total value) that each sales support agent has sold and the number of customers that each sales support agent has.

In [7]:
%%sql

/* #Grouping sales by "InvoiceId" because "InvoiceId" is PRIMARY KEY */

WITH reps_invoices AS 

(
    
    SELECT 
        
        e.FirstName || " " || e.LastName "Support Rep",
        
        i.InvoiceId "Invoice Id",
        i.Total
            
            FROM employees e
            
            INNER JOIN customers c ON c.SupportRepId = e.EmployeeId
            INNER JOIN invoices i ON i.CustomerId = c.CustomerId
            
                WHERE e.Title = "Sales Support Agent"
            
                    GROUP BY 2
    
)

/* #Grouping by sales support agent and summing "Total" column */

SELECT 

    "Support Rep", 
    
    SUM(Total) "Total Sold" 
        
        FROM reps_invoices
    
            GROUP BY 1;

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


Support Rep,Total Sold
Jane Peacock,833.0400000000016
Margaret Park,775.4000000000005
Steve Johnson,720.1600000000011


In [8]:
%%sql

/* #Calculating number of customers per sales support agent */

SELECT
    
    e.FirstName || " " || e.LastName "Support Rep",
    
    COUNT(DISTINCT c.CustomerId) "Number of Customers"
        
        FROM employees e
        
        INNER JOIN customers c ON c.SupportRepId = e.EmployeeId
        INNER JOIN invoices i ON i.CustomerId = c.CustomerId
        
            WHERE e.Title = "Sales Support Agent"
        
                GROUP BY 1;

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


Support Rep,Number of Customers
Jane Peacock,21
Margaret Park,20
Steve Johnson,18


### Answer
Jane Peacock has sold the most in terms of value (833 dollars) and Jane Peacock has the most customers (21 customers). Steve Johnson has sold the least in terms of value (720 dollars) and Steve Johnson has the least amount of customers (18 customers).

However, none of the agents display outlier behavior, and all are more or less performing equally. Margaret Park's performance is in the middle of the other two, having sold 775 dollars worth of value and having 20 customers. All agents have roughly the same amount of customers and still roughly the same amount of sales in terms of value.

### Calculate for each country: (a) the total number of customers, (b) the total value of sales, (c) the average value of sales per customer, (d) and the average order value.

In [9]:
%%sql

/* #Calculating aggregate data */

SELECT

    c.Country,
    
    COUNT(DISTINCT c.CustomerId) "Number of Customers",
    
    ROUND(SUM(i.Total),2) "Total Value of Sales",
    
    ROUND(CAST(SUM(i.Total) AS Float) / COUNT(DISTINCT c.CustomerId),2) "Average Value of Sales per Customer",
    ROUND(CAST(SUM(i.Total) AS Float) / COUNT(DISTINCT i.InvoiceId), 2) "Average Order Value"
        
        FROM customers c
        
        INNER JOIN invoices i ON i.CustomerId = c.CustomerId
        
            GROUP BY c.Country
        
            ORDER BY 3 DESC;

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


Country,Number of Customers,Total Value of Sales,Average Value of Sales per Customer,Average Order Value
USA,13,523.06,40.24,5.75
Canada,8,303.96,38.0,5.43
France,5,195.1,39.02,5.57
Brazil,5,190.1,38.02,5.43
Germany,4,156.48,39.12,5.59
United Kingdom,3,112.86,37.62,5.37
Czech Republic,2,90.24,45.12,6.45
Portugal,2,77.24,38.62,5.52
India,2,75.26,37.63,5.79
Chile,1,46.62,46.62,6.66


This is a good table that completes the task, but there are a couple concerns that I have about the data that I will address before moving on.

First of all, it is strange that nearly all customers seem to have exactly 7 orders. This is a mock database, so the data is fictitious. Still, I will perform a sanity check to guarantee that this data is accurate.

Second of all, many countries only have 1 customer in them. To feature prominently those countries that have more than 1 customer in this report, and to make the more valuable data in this dataset more easily accessible, I will group all countries with only 1 customer into one category, "Other."

In [10]:
%%sql

/* #Sanity checking the 7-order-per-customer phenomenon */

SELECT 
    
    c.FirstName || " " || c.LastName Name,
    c.Country,
    
    COUNT(DISTINCT i.InvoiceId) "Number of Orders"
        
        FROM customers c 
        
        INNER JOIN invoices i ON i.CustomerId = c.CustomerId
        
            GROUP BY c.CustomerId
        
                HAVING "Number of Orders" <> 7;

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


Name,Country,Number of Orders
Puja Srivastava,India,6


Yes, apparently every customer in this fictitious database has exactly 7 orders placed, except for one customer from India, Puja Srivastava, who has 6.

In [11]:
%%sql

WITH 

    sales_summary AS (                      /* #Summarizing the sales data with "Other Categorization" column */

        SELECT
        
            c.Country,
        
            COUNT(DISTINCT i.InvoiceId)"Number of Orders",
            COUNT(DISTINCT c.CustomerId) "Number of Customers",
        
            ROUND(SUM(i.Total),2) "Total Value of Sales",
            
            ROUND(CAST(SUM(i.Total) AS Float) / COUNT(DISTINCT c.CustomerId),2) "Average Value of Sales per Customer",
            ROUND(CAST(SUM(i.Total) AS Float) / COUNT(DISTINCT i.InvoiceId), 2) "Average Order Value",
        
            CASE 
                
                WHEN COUNT(DISTINCT c.CustomerId) = 1 THEN "Other"
                WHEN COUNT(DISTINCT c.CustomerId) > 1 THEN c.Country
            
            END "Other Categorization"
            
                FROM customers c
            
                INNER JOIN invoices i ON i.CustomerId = c.CustomerId
            
                    GROUP BY c.Country
            
                    ORDER BY 4 DESC

    ),

    summary_sort AS (                        /* #Sorting sales_summary by the "Other Categorization" column */
            
        SELECT *,
                
            CASE 
                
                WHEN "Other Categorization" = "Other" THEN 0
                ELSE 1
            
            END sort
            
                FROM sales_summary
        
)


SELECT
    
    "Other Categorization" Country,
    
    SUM("Number of Customers") "Number of Customers",
    SUM("Total Value of Sales") "Total Value of Sales",
    
    ROUND(CAST(SUM("Total Value of Sales") AS Float) / SUM("Number of Customers"),2) "Average Value of Sales per Customer",
    ROUND(CAST(SUM("Total Value of Sales") AS Float) / SUM("Number of Orders"), 2) "Average Order Value"
    
        FROM summary_sort
        
            GROUP BY "Other Categorization"
        
            ORDER BY sort DESC, 3 DESC;

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


Country,Number of Customers,Total Value of Sales,Average Value of Sales per Customer,Average Order Value
USA,13,523.06,40.24,5.75
Canada,8,303.96,38.0,5.43
France,5,195.1,39.02,5.57
Brazil,5,190.1,38.02,5.43
Germany,4,156.48,39.12,5.59
United Kingdom,3,112.86,37.62,5.37
Czech Republic,2,90.24,45.12,6.45
Portugal,2,77.24,38.62,5.52
India,2,75.26,37.63,5.79
Other,15,604.3,40.29,5.76


This table is much easier to read since, generally, our business prefers to focus on countries with more customers and a greater total value of sales.

The USA has the highest number of customers and total value sold, with 13 customers and 523.06 dollars of value sold respectively. Trailing behind the USA, in terms of number of customers and total value of sales, are Canada, France, Brazil, and Germany, in that order. Three countries have the lowest number of customers, Czech Republic, Porgual, and India, all with 2 customers. Their total value of sales are all the lowest as well, ranging from 75.26 dollars to 90.24 dollars.

There is a direct relationship, a positive correlation, between the number of customers that a country has and the total value of sales for that country. The greater the number of customers a country has, the higher that country's total value of sales is. This is true even when considering the "Other category," which has 15 customers and 604.30 dollars in total value sold. Both numbers are higher than the leading individual country, the USA.

The Czech Republic has the highest average values, in terms of average value of sales per customer and average order value, with 45.12 dollars and 6.45 dollars respectively. Most other countries, including the "Other" category, have an average value of sales per customer hovering around 38.00 to 40.00 dollars per customer and an average order value of approximately 5.50 dollars. The two countries with the lowest average value of sales per customer are India and the United Kingdom, with 37.63 and 37.62 dollars per customer respectively. The country with the lowest average order value is the United Kingdom, with 5.37 dollars per order.

### What percentage of purchases are individual tracks vs. whole albums?
Our company is currently considering a new purchasing strategy. Should we only purchase popular tracks from albums rather than purcahsing the full albums for our online store?

To answer this question, I will determine how popular whole-album purcahsing is for our customers. If a large percentage of our sales (greater than 40%) are full-album purchases, then it may be wise to purchase full albums rather than only popular tracks.

In [12]:
%%sql

WITH 

    album_tracks AS (               /* #Generating table of albums and their corresponding number of tracks */

    SELECT

        a.AlbumId "Album",
    
        COUNT(TrackId) "Number of Tracks"

            FROM albums a
    
            INNER JOIN tracks t ON t.AlbumId = a.AlbumId
    
                GROUP BY a.AlbumId
    
    ),

    single_album_purchases AS (     /* #Generating table of all 1-album purchases that are not 1-track purchases */

        SELECT

            i.InvoiceId "Invoice",
    
            a.AlbumId "Album",
    
            COUNT(ii.TrackId) "Number of Songs",
    
            COUNT(DISTINCT t.AlbumId) "Number of Different Albums"

                FROM invoices i
    
                INNER JOIN invoice_items ii ON ii.InvoiceId = i.InvoiceId
                INNER JOIN tracks t ON t.TrackId = ii.TrackId
                INNER JOIN albums a ON a.AlbumId = t.AlbumId
                       
                    GROUP BY i.InvoiceId
    
                        HAVING "Number of Different Albums" = 1
                        
                        AND "Number of Songs" <> 1

    )
    
SELECT COUNT(Invoice) "Number of Whole Album Purchases"     /* #Determining the number of whole purchases */

    FROM single_album_purchases sap
    
    INNER JOIN album_tracks at ON at.Album = sap.Album
    
        WHERE "Number of Songs" = "Number of Tracks"

 * sqlite:////Users/admin/Downloads/Databases/chinook.db
Done.


Number of Whole Album Purchases
0


### Answer
None of the sales made in this mock set are whole album purchases. This answer is anticlimactic, and different from the result in the solution manual for this project <a href='https://github.com/dataquestio/solutions/blob/master/Mission191Solutions.ipynb'>here</a>. In fact, many of my answers differ from the solution manual. Upon further inspection, the Chinook database that I used is different from the one that was used in the official solution guide. This is evidenced by the number of invoices in each database (mine has 412, the one in the solution guide has 614), the "Hire Dates" of the employees, etc.

After completing the project using this database, and because the template for this project does not specify which Chinook database to use (or where to find it online), I will accept this answer and finish the project as such.

Therefore, it is best that our company only purchases popular tracks from our record label partners. All of our customers only purchase songs track-by-track for now.

## Conclusion
The three albums that we should include from our new record label deal are Red Tone's punk album, Slim Jim Bites' blues album, and Meteor and the Girls' pop album.

All of the sales support representatives in our company are performing well compared to one another in terms of total sales value and number of sales.

The USA has the largest share of our music media market, with Canada, France, Brazil, and Germany trailing a good distance behind.

Our management team should focus on purchasing only hit popular songs in our record deals since all of the track purchases in our store are only individual track purchases and not full-album purchases.